In [106]:
from selenium import webdriver 
import time 
import urllib.parse
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = webdriver.ChromeOptions(); 
chrome_options.add_experimental_option("excludeSwitches", ['enable-automation']);
# driver = webdriver.Chrome();  
driver = webdriver.Chrome(executable_path=r'chromedriver',options=chrome_options)

## Scrape travel insurance products!

In [107]:
driver.get("https://www.gobear.com/sg/travel-insurance")
countries = []
time.sleep(5)
select_box = driver.find_element_by_name("travel-form-country") 
options = [x for x in select_box.find_elements_by_tag_name("option")]
for element in options:
    countries.append(element.get_attribute("value"))
countries.remove('---------------------------------')
productbyCountry = {}

In [108]:
count = 0
start_time = time.time()
for country in countries:
    country_parsed = urllib.parse.quote(country)
    try:
        driver.get(f'https://www.gobear.com/sg/insurance/travel/quote-online?triptype=single&covertype=individual&destination={country_parsed}&adults=1&startdate=2020-10-30&enddate=2020-11-01&selectedCoverageOptionIDs=&gender=&age=')
    except:
        continue
        
    time.sleep(30) 
    try:
        number = driver.find_element_by_xpath('//*[@id="wrapper"]/nav/div/div[1]/h5')
        products = int(number.text.partition(' ')[0])
        if products == 0:
            continue
    except:
        continue
        
    for i in range(1,products+1):
        product = {}
        var = driver.find_element_by_xpath(f'//*[@id="travel-quote-list"]/div[2]/div/div[{i}]')
        company_name = var.find_element_by_class_name("name")
        product['Insurance Company'] = company_name.text
        # print(company_name.text)
        
        policy_name = var.find_element_by_class_name("plan-name")
        product['Policy Name'] = policy_name.text
        # print(policy_name.text)
        
        cost = var.find_element_by_class_name("value")
        product['Cost'] = cost.text
        # print(cost.text)
        
        review = var.find_element_by_class_name("coverage-score")
        product['Review'] = review.text
        # print(review.text)
        
        props = var.find_elements_by_class_name("col-xs-12")
        props_info = {}
        for prop in props:
            key = prop.find_element_by_class_name("detail-key")
            val = prop.find_element_by_class_name("detail-value")
            props_info[key.text] = val.text
            
            # print(f'{key.text}: {val.text}')   
        product['Properties'] = props_info
        country_products[i] = product
    productbyCountry[country] = country_products
    count += 1
    elapsed = time.time() - start_time
    elapsed_min = elapsed/60
    print(f'Scrapped country: {country}. Completed {count}/255...')
    print(f'{elapsed_min} minutes elapsed')
    with open('data.json', 'w') as json_file:
        json.dump(productbyCountry, json_file, indent=1)
        json_file.close()
    

Scrapped country: Thailand. Completed 1/255...
0.8584766228993733 minutes elapsed
Scrapped country: Malaysia. Completed 2/255...
1.7539213021596274 minutes elapsed
Scrapped country: Japan. Completed 3/255...
2.570231072107951 minutes elapsed
Scrapped country: Taiwan. Completed 4/255...
3.374168054262797 minutes elapsed
Scrapped country: Hong Kong. Completed 5/255...
4.178912750879923 minutes elapsed
Scrapped country: China. Completed 6/255...
4.974001320203145 minutes elapsed
Scrapped country: Australia. Completed 7/255...
6.026367203394572 minutes elapsed
Scrapped country: Indonesia. Completed 8/255...
6.943902540206909 minutes elapsed
Scrapped country: South Korea. Completed 9/255...
7.984938351313273 minutes elapsed
Scrapped country: United Kingdom. Completed 10/255...
8.930592254797618 minutes elapsed
Scrapped country: Vietnam. Completed 11/255...
9.960380438963572 minutes elapsed
Scrapped country: Philippines. Completed 12/255...
10.956706770261128 minutes elapsed
Scrapped country

KeyboardInterrupt: 

## Scrape maid insurance products

In [104]:
# driver = webdriver.Chrome(executable_path=r'chromedriver')
driver.get("https://www.gobear.com/sg/lp/insurance/maid-insurance?filter=%7B%22providers%22%3A%5B%5D%2C%22c1b8d263-8432-4184-90d0-1a5261983295%22%3A%5B%2226%20months%22%5D%7D&sort=%7B%22id%22%3A%2258e4b5f1-f494-4a25-97e9-9646fa61c66c%22%2C%22sortBy%22%3A%2219e67179-71f6-4261-8b91-4bcb6158030b%22%2C%22order%22%3A%22Asc%22%7D")
maid_products = {}
time.sleep(5)
var = driver.find_elements_by_class_name("list-card-wrap")
properties = ['Personal Accident', 'Hospital & Surgical Expenses', 'Yearly Premium']
for i,element in enumerate(var):
    product = {}
    name = element.find_element_by_class_name("name-wrap")
    product["Name"] = name.text
    personal = element.find_elements_by_class_name("value")
    props = {}
    for j in range(len(personal)):
        props[properties[j]] = personal[j].text
    product["Coverage"] = props
    maid_products[i] = product

with open('data_maidInsurance.json', 'w') as json_file:
    json.dump(maid_products, json_file, indent=2)
    json_file.close()

In [100]:
## Scrape maid insurance products

In [105]:
maid_products

{0: {'Name': 'MSIG MaidPlus - Standard (26 months)',
  'Coverage': {'Personal Accident': '$60,000',
   'Hospital & Surgical Expenses': '$15,000',
   'Yearly Premium': '$214.00'}},
 1: {'Name': 'FWD Maid Insurance - Essential (26 months)',
  'Coverage': {'Personal Accident': '$60,000',
   'Hospital & Surgical Expenses': '$15,000',
   'Yearly Premium': '$224.27'}},
 2: {'Name': 'FWD Maid Insurance - Enhanced (26 months)',
  'Coverage': {'Personal Accident': '$60,000',
   'Hospital & Surgical Expenses': '$20,000',
   'Yearly Premium': '$268.22'}},
 3: {'Name': 'Great Eastern MaidGR8 - Plan B (26 months)',
  'Coverage': {'Personal Accident': '$60,000',
   'Hospital & Surgical Expenses': 'Up to $15,000',
   'Yearly Premium': '$278.20'}},
 4: {'Name': 'MSIG MaidPlus - Classic (26 months)',
  'Coverage': {'Personal Accident': '$60,000',
   'Hospital & Surgical Expenses': '$30,000',
   'Yearly Premium': '$291.04'}},
 5: {'Name': 'FWD Maid Insurance - Exclusive (26 months)',
  'Coverage': {'Per

In [ ]:
## Scrape personal accidents insurance products

In [130]:
# driver = webdriver.Chrome(executable_path=r'chromedriver')
driver.get("https://www.gobear.com/sg/personal-accident-insurance")
time.sleep(10)
var = driver.find_elements_by_class_name("panel-body")
accident_products = {}
pr
# for i,product in enumerate(var):
#     product_dict = {}
#     desc = product.text.split('\n')
#     product_dict['Name'] = desc[0]
#     product_dict['Cost'] = desc[1]
#     props = {}
#     for line in desc:
#         if ':' in line:
#             index = line.index(':')
#             key = line[:index]
#             val = line[index+1:]
#             props[key] = val
#     product_dict['Properties'] = props
#     accident_products[i] = product_dict

Accidental Death or TPD:  $100,000
Accident Related Medical Expenses:  $3,000
TCM or Chiropractor Treatment (as a sub-limit of accident medical expenses):  $500
Daily Hospital Allowance              (up to 30 days):  $50/day
Personal Liability:   $100,000
Weekly Temporary Disablement Benefit (up to 104 weeks):   up to $200/week
Mobility Aid & Ambulance Services Reimbursement:   $2,000
Accidental Death or TPD:  $100,000
Accident Related Medical Expenses:  $3,000
TCM or Chiropractor Treatment (as a sub-limit of accident medical expenses):  $500
Daily Hospital Allowance              (up to 30 days):  $50/day
Personal Liability:   $100,000
Weekly Temporary Disablement Benefit (up to 104 weeks):   up to $200/week
Mobility Aid & Ambulance Services Reimbursement:   $2,000
Accidental Death or TPD:  $100,000
Accident Related Medical Expenses:  $1,000
TCM Treatment:  $1,000
Daily Hospital Allowance (up to 50 days):  $50/day
Mobility Aid:  $1,000
Accidental Death or TPD:  $100,000
Accident Relate

In [132]:
with open('data_accidentInsurance.json', 'w') as json_file:
    json.dump(accident_products, json_file, indent=2)
    json_file.close()